In [1]:
#ecg_wave_model.py
import os
import sys

from keras.layers import Lambda, Input, Dense, LSTM, RepeatVector, concatenate
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.losses import mse, binary_crossentropy, kullback_leibler_divergence
from keras.utils import plot_model
from keras import backend as K
from keras.regularizers import l1
import numpy as np
from keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, Lambda
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten
from keras.layers.convolutional import Conv1D, Conv2D, Conv3D, UpSampling2D, UpSampling3D, MaxPooling1D
from keras.layers.convolutional import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D

# IPython imports
from IPython.display import Image

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.tensor_generators import test_train_valid_tensor_generators
from ml4cvd.defines import TENSOR_EXT, IMAGE_EXT, MRI_TO_SEGMENT, MRI_SEGMENTED
from ml4cvd.recipes import train_multimodal_multitask, test_multimodal_multitask
from ml4cvd.models import make_multimodal_to_multilabel_model, train_model_from_generators

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [2]:
input_x = Input(shape=(5000, 12), name='in')
x = Conv1D(filters=128, kernel_size=101, activation='relu', padding='same')(input_x)
x = Conv1D(filters=128, kernel_size=101, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=51, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=51, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
conv_label = Conv1D(12, 1, activation="linear", padding='same')(x)
output_y = Activation('linear', name='out')(conv_label)
m = Model(inputs=[input_x], outputs=[output_y])
m.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m.compile(optimizer=opt, loss='logcosh')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (InputLayer)              (None, 5000, 12)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5000, 128)         155264    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4900, 128)         1654912   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2450, 128)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2400, 128)         835712    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1200, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1200, 128)         835712    
__________

In [3]:
input_x = Input(shape=(5000, 12), name='in')
x = Conv1D(filters=64, kernel_size=101, activation='relu', padding='same')(input_x)
x = Conv1D(filters=64, kernel_size=101, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='valid')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=51, activation='relu', padding='same')(x)
conv_label = Conv1D(12, 1, activation="linear", padding='same')(x)
output_wave = Activation('linear', name='out')(conv_label)

alphabet_size = 60
window_size = 40

flat_x = Flatten()(x)
flat_x = Dense(16, activation='relu')(flat_x)
wave_embeds = RepeatVector(window_size)(flat_x)
#Permute((2, 1)

burn_in = Input(shape=(window_size, alphabet_size), name='burn_in')
lstm_in = concatenate([burn_in, wave_embeds])

lstm_out = LSTM(128)(lstm_in)
output_char = Dense(alphabet_size, activation='softmax')(lstm_out)

m = Model(inputs=[input_x, burn_in], outputs=[output_wave, output_char])
m.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m.compile(optimizer=opt, loss=['logcosh', 'categorical_crossentropy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in (InputLayer)                 (None, 5000, 12)     0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 5000, 64)     77632       in[0][0]                         
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 4900, 64)     413760      conv1d_6[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 2450, 64)     0           conv1d_7[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (

In [ ]:
sys.argv = ['train', 
            '--tensors','/mnt/disks/ecg-rest-31k/2019-06-10/', 
            '--input_tensors', 'ecg_rest_1lead', 
            '--output_tensors','ecg_median_1lead', 'ecg_rhythm', 'ecg_normal', 'ventricular-rate', 
            '--batch_size', '32', 
            '--epochs', '1',
            '--training_steps', '30',
            '--inspect_model',
            '--u_connect',
            '--id', 'ecg_wave_regress_and_classify']
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,
                                                                                   args.tensor_maps_out,
                                                                                   args.tensors,
                                                                                   args.batch_size,
                                                                                   args.valid_ratio,
                                                                                   args.test_ratio,
                                                                                   args.test_modulo,
                                                                                   args.balance_csvs)

2019-07-29 21:33:45,637 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_wave_regress_and_classify/log_2019-07-29_21-33_0.log.
2019-07-29 21:33:45,638 - arguments:180 - INFO - Command Line was:

python train --tensors /mnt/disks/ecg-rest-31k/2019-06-10/ --input_tensors ecg_rest_1lead --output_tensors ecg_median_1lead ecg_rhythm ecg_normal ventricular-rate --batch_size 32 --epochs 1 --training_steps 30 --inspect_model --u_connect --id ecg_wave_regress_and_classify


2019-07-29 21:33:45,639 - arguments:181 - INFO - Total TensorMaps:4410 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=32, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, categorical_field_ids=[], continuous_field_ids=[], conv_bn=False, conv_dropout=0.0, conv_layers=[3

In [ ]:
model = make_multimodal_to_multilabel_model(args.model_file, args.model_layers, args.model_freeze, args.tensor_maps_in, args.tensor_maps_out,
                                            args.activation, args.dense_layers, args.dropout, args.mlp_concat, args.conv_layers, args.max_pools,
                                            args.res_layers, args.dense_blocks, args.block_size, args.conv_bn, args.conv_x, args.conv_y,
                                            args.conv_z, args.conv_dropout, args.conv_width, args.u_connect, args.pool_x, args.pool_y,
                                            args.pool_z, args.padding, args.learning_rate)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_strip_ecg_rest (InputLaye (None, 600, 8)       0                                            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 600, 32)      18208       input_strip_ecg_rest[0][0]       
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 600, 32)      72736       conv1d_12[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 300, 32)      0           conv1d_13[0][0]                  
__________________________________________________________________________________________________
conv1d_15 

In [ ]:
model = train_model_from_generators(model, generate_train, generate_valid, args.training_steps, args.validation_steps, args.batch_size,
                                    args.epochs, args.patience, args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

2019-07-29 21:33:46,879 - models:852 - INFO - Saving architecture diagram to:./recipes_output/ecg_wave_regress_and_classify/architecture_graph_ecg_wave_regress_and_classify.png
Epoch 1/1
2019-07-29 21:33:54,083 - tensor_generators:373 - INFO - Got first error: KeyError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 80, in multimodal_multitask_generator
    in_batch[tm.input_name()][stats['batch_index']] = tm.tensor_from_file(hd5, dependents)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/TensorMap.py", line 594, in tensor_from_file
    dependents[self.dependent_map][:, 0] = np.array(hd5[self.group][lead_idx.replace(self.name, self.dependent_map.name)])
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/usr/local/lib/python3.6/dist-packages/h5py/_hl/group.py", line 262, in _

In [ ]:
Image('./recipes_output/ecg_wave_regress_and_classify/architecture_graph_ecg_wave_regress_and_classify.png')

In [ ]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('embed').output)
intermediate_layer_model.summary()
intermediate_output = intermediate_layer_model.predict_generator(generate_test, steps=1)
print(intermediate_output.shape)

In [ ]:
embed = model.get_layer('embed').output

burn_in = Input(shape=(window_size, alphabet_size), name='burn_in')
embed_in = Input(shape=(64,), name='embed_in')
wave_embeds = RepeatVector(window_size)(embed_in)
lstm_in = concatenate([burn_in, wave_embeds])
alphabet_size = 60
window_size = 40
lstm_out = LSTM(128)(lstm_in)
output_char = Dense(alphabet_size, activation='softmax')(lstm_out)

m_caption = Model(inputs=[embed_in, burn_in], outputs=[output_char])
m_caption.summary()
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m_caption.compile(optimizer=opt, loss='categorical_crossentropy')

In [ ]:
Image('./recipes_output/ecg_wave_regress_and_classify/architecture_graph_ecg_wave_regress_and_classify.png')